In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astropy import units as u
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time

from astroplan.plots import plot_airmass, plot_sky, plot_sky_24hr, plot_finder_image
from matplotlib import pyplot as plt
%matplotlib inline

from astroplan import Observer

from pocs.scheduler.dispatch import Scheduler
from pocs.scheduler.constraint import Duration, MoonAvoidance
from pocs.utils.config import load_config
from pocs.utils import listify

config = load_config()

In [3]:
# For VOparser

from __future__ import print_function
import copy
import voeventparse
from voeventparse.tests.resources.datapaths import swift_bat_grb_pos_v2

with open(swift_bat_grb_pos_v2, 'rb') as f:
    v = voeventparse.load(f)

In [4]:
print("Ivorn:", v.attrib['ivorn'])
print("Role:", v.attrib['role'])
print( "AuthorIVORN:", v.Who.AuthorIVORN)
print( "Short name:", v.Who.Author.shortName)
print( "Contact:", v.Who.Author.contactEmail)

Ivorn: ivo://nasa.gsfc.gcn/SWIFT#BAT_GRB_Pos_532871-729
Role: observation
AuthorIVORN: ivo://nasa.gsfc.tan/gcn
Short name: VO-GCN
Contact: scott.barthelmy@nasa.gov


In [5]:
c = voeventparse.pull_astro_coords(v)
t = voeventparse.pull_isotime(v)
print( "Coords:", c)
print(t)

Coords: Position2D(ra=74.7412, dec=-9.3137, err=0.05, units='deg', system='UTC-FK5-GEO')
2012-09-07 00:24:23.080000+00:00


In [6]:
# returns string in desired format for POCS - converts decimal VO value to DegMinSec format
# other method converts time, so that it does not have as many dec places - this was an issue when pocs 
# ran the sunrise/ sunset time.

def convert_time(tim):
    time = str(tim)
    return time[0:19]

def convert_to_deg(num, typ):
    _h = int(num)
    _m=(num-_h)*60
    _min = int(_m)
    _s=round((_m-_min)*60,3)
    result = str(_h)+typ+str(_min)+"m"+str(_s)+"s"
    
    return result

In [7]:
ra = convert_to_deg(c[0], "h") #<- RA has first unit of hours, so we give the method the first unit in the string
dec = convert_to_deg(c[1],"d")

time = convert_time(t)

coords = [ra, dec]
print(coords) #<- checking that it works
print(time)

['74h44m28.32s', '-9d-18m-49.32s']
2012-09-07 00:24:23


In [11]:
# Get a location
loc = config['location']
location = EarthLocation(lon=loc['longitude'], lat=loc['latitude'], height=loc['elevation'])

# Get an observer at the location
observer = Observer(location=location)

# Get the night hours (you can use observer.tonight())
#times = observer.tonight(horizon=-18 * u.degree)
#start_of_night = times[0]
#end_of_night = times[1]
start_of_night = Time('2016-08-13 10:00:00')
end_of_night = Time('2016-08-13 16:00:00')

In [12]:
# Look up relevant objects at given coords


In [13]:
# Creating new config .yaml file to store our observation

new_confi = {'name': 'ToO', 'position': coords[0]+" "+coords[1], 'exp_time': '10', 'priority': '100000'}

In [14]:
# Here I want to create a new observation
simple_fields_file = config['directories']['targets'] + '/simple.yaml'

# Simple constraint to maximize duration above a certain altitude
constraints = [MoonAvoidance(), Duration(30 * u.deg)]

# Create a scheduler
scheduler = Scheduler(observer, fields_file=simple_fields_file, constraints=constraints)

scheduler.add_observation(new_confi)

ValueError: ('Cannot parse first argument data "74h44m28.32s" for attribute ra', IllegalHourError(74.0,))